In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from numpy.linalg import eig

In [2]:
path = "C:/Users/Sachith/Desktop/work_oh/bulk_water/MD1"

In [3]:
f=open(path +'/input_data.dat','r')
lines=f.readlines()
for line in lines[0:]:
    data=line.split()
    if data[0]=='nsteps:':
        nsteps = int(data[1])
    if data[0]=='natoms:':
        natoms = int(data[1])
    if data[0]=='nonmvat:':
        nonmvat = int(data[1])
    if data[0]=='nmovingatoms:':
        nmovingatoms = int(data[1])
    if data[0]=='MDRestartFrequency:':
        MDRestartFrequency = int(data[1])
    if data[0]=='dt:':
        dt = float(data[1])
    if data[0]=='noh:':
        noh = int(data[1])
    if data[0]=='nwater:':
        nwater = int(data[1])
    if data[0]=='nox:':
        nox = int(data[1])
    if data[0]=='nhy:':
        nhy = int(data[1])
    if data[0]=='xbox:':
        xbox = float(data[1])
    if data[0]=='ybox:':
        ybox = float(data[1])
    if data[0]=='zbox:':
        zbox = float(data[1])
        
nsteps = int((nsteps/MDRestartFrequency)+1)-1
nstep = nsteps
        
print('nsteps:','  ',nstep )
print('natoms:','  ',natoms)
print('nonmvat:','  ',nonmvat)
print('nmovingatoms:','  ',nmovingatoms)
print('MDRestartFrequency:','  ',MDRestartFrequency)
print('dt:','  ',dt)
print('noh:','  ',noh)
print('nwater:','  ',nwater)
print('nox:','  ',nox)
print('nhy:','  ',nhy)
print('xbox:','  ',xbox)
print('ybox:','  ',ybox)
print('zbox:','  ',zbox)

nsteps:    20000
natoms:    654
nonmvat:    432
nmovingatoms:    222
MDRestartFrequency:    1
dt:    1.0
noh:    0
nwater:    74
nox:    74
nhy:    148
xbox:    22.784
ybox:    13.3552
zbox:    40.0


In [4]:
msd_of_oh = np.empty(nstep)
msd_of_ohx = np.empty(nstep)
msd_of_ohy = np.empty(nstep)
msd_of_ohz = np.empty(nstep)
oh_index = np.empty(nstep)
xoh = np.empty(nstep)
yoh = np.empty(nstep)
zoh = np.empty(nstep)
atom= ['Non']*(natoms)
x_ox = np.empty(nstep*nox).reshape(nox,nstep)
y_ox = np.empty(nstep*nox).reshape(nox,nstep)
z_ox = np.empty(nstep*nox).reshape(nox,nstep)

x_hy = np.empty(nstep*nhy).reshape(nhy,nstep)
y_hy = np.empty(nstep*nhy).reshape(nhy,nstep)
z_hy = np.empty(nstep*nhy).reshape(nhy,nstep)

In [5]:
jj=0
kk=0
f=open(path +'/geo_end.xyz','r')
lines=f.readlines()
while jj < nstep:
    io=0
    ih=0
    for line in lines[kk+nonmvat+2:(kk+natoms+2)]:
        data=line.split()
        if len(data)>3:
            if data[0]=='O':
                x_ox[io][jj]=float(data[1])
                y_ox[io][jj]=float(data[2])
                z_ox[io][jj]=float(data[3])
                io = io+1
            elif data[0]=='H':
                x_hy[ih][jj]=float(data[1])
                y_hy[ih][jj]=float(data[2])
                z_hy[ih][jj]=float(data[3])
                ih = ih+1
    kk=kk+natoms+2
    jj=jj+1
f.close()

In [6]:
nrep = 24
nsteps = 1000
aa = [24,42,49,63,18,22,5,25,69,26,20,10,71,1,39,68,51,8,19,55,64,9,60,52]
x_o = np.empty(nsteps*nrep).reshape(nrep,nsteps)
y_o = np.empty(nsteps*nrep).reshape(nrep,nsteps)
z_o = np.empty(nsteps*nrep).reshape(nrep,nsteps)

In [7]:
kk = 0
while (kk<nrep):
    jj = 0
    while (jj<1000):
        x_o[kk][jj] = x_ox[aa[kk]][jj*10]
        y_o[kk][jj] = y_ox[aa[kk]][jj*10]
        z_o[kk][jj] = z_ox[aa[kk]][jj*10]
        jj = jj+1
    kk = kk+10


In [8]:
ndt = 50
mmsdx = np.empty(ndt)
mmsdy = np.empty(ndt)
mmsdz = np.empty(ndt)
mmsdxy = np.empty(ndt)
mmsdxz = np.empty(ndt)
mmsdyz = np.empty(ndt)
dtime = np.empty(ndt)
dtime=  np.arange(0.01, (0.01*ndt+0.01), 0.01)

Dfxx = np.empty(nrep)  
Dfyy = np.empty(nrep)
Dfzz = np.empty(nrep)
Dfxy = np.empty(nrep)
Dfxz = np.empty(nrep)
Dfyz = np.empty(nrep)
Dfx2d = np.empty(nrep)
Dfy2d = np.empty(nrep)
Df2d = np.empty(nrep)

ll = 0
while ll<nrep:
    kk = 0
    while kk<ndt:
        mmsdx[kk] = 0.0
        mmsdy[kk] = 0.0
        mmsdz[kk] = 0.0
        mmsdxy[kk] = 0.0
        mmsdxz[kk] = 0.0
        mmsdyz[kk] = 0.0
        jj = 0
        while jj< ((nsteps/(kk+1))-1):
            mmsdx[kk] = (x_o[ll,(jj*(kk+1)+kk+1)]-x_o[ll,jj*(kk+1)])**2+mmsdx[kk]
            mmsdy[kk] = (y_o[ll,(jj*(kk+1)+kk+1)]-y_o[ll,jj*(kk+1)])**2+mmsdy[kk]
            mmsdz[kk] = (z_o[ll,(jj*(kk+1)+kk+1)]-z_o[ll,jj*(kk+1)])**2+mmsdz[kk]
            mmsdxy[kk] = ((x_o[ll,(jj*(kk+1)+kk+1)]-x_o[ll,jj*(kk+1)])*(y_o[ll,(jj*(kk+1)+kk+1)]-y_o[ll,jj*(kk+1)]))+mmsdxy[kk]
            mmsdxz[kk] = ((x_o[ll,(jj*(kk+1)+kk+1)]-x_o[ll,jj*(kk+1)])*(z_o[ll,(jj*(kk+1)+kk+1)]-z_o[ll,jj*(kk+1)]))+mmsdxz[kk]
            mmsdyz[kk] = ((y_o[ll,(jj*(kk+1)+kk+1)]-y_o[ll,jj*(kk+1)])*(z_o[ll,(jj*(kk+1)+kk+1)]-z_o[ll,jj*(kk+1)]))+mmsdyz[kk]
            
            jj = jj+1
    
        mmsdx[kk] = mmsdx[kk]/(((nsteps/(kk+1))-1))
        mmsdy[kk] = mmsdy[kk]/(((nsteps/(kk+1))-1))
        mmsdz[kk] = mmsdz[kk]/(((nsteps/(kk+1))-1))
        mmsdxy[kk] = mmsdxy[kk]/(((nsteps/(kk+1))-1))
        mmsdxz[kk] = mmsdxz[kk]/(((nsteps/(kk+1))-1))
        mmsdyz[kk] = mmsdyz[kk]/(((nsteps/(kk+1))-1))
        print(ll, mmsdx[kk],nsteps,(kk+1),(((nsteps/(kk+1))-1)))
        kk = kk+1
    
    
    fitx=np.polyfit(dtime[:],mmsdx[:],1)
    fity=np.polyfit(dtime[:],mmsdy[:],1)
    fitz=np.polyfit(dtime[:],mmsdz[:],1)
    fitxy=np.polyfit(dtime[:],mmsdxy[:],1)
    fitxz=np.polyfit(dtime[:],mmsdxz[:],1)
    fityz=np.polyfit(dtime[:],mmsdyz[:],1)

    dxx = fitx[0]/2
    dyy = fity[0]/2
    dzz = fitz[0]/2
    dxy = fitxy[0]/2
    dxz = fitxz[0]/2
    dyz = fityz[0]/2


    dd2d = np.array([[dxx, dxy], 
                 [dxy, dyy]])
    w2,v2=eig(dd2d)


    Dfxx[ll] = dxx
    Dfyy[ll] = dyy
    Dfzz[ll] = dzz
    Dfxy[ll] = dxy
    Dfxz[ll] = dxz
    Dfyz[ll] = dyz
    Dfx2d[ll] = w2[0]
    Dfy2d[ll] = w2[1]
    Df2d[ll] = ((w2[0]+w2[1])/2)
    
#    print(ll ,Dfxx[ll])
    ll = ll+1



0 0.0020509832053246598 1000 1 999.0
0 0.007854070193950353 1000 2 499.0
0 0.01688213352879013 1000 3 332.3333333333333
0 0.027932233282990757 1000 4 249.0
0 0.04106250396446527 1000 5 199.0
0 0.05582219296497697 1000 6 165.66666666666666
0 0.06921077199826235 1000 7 141.85714285714286
0 0.07991331958665 1000 8 124.0
0 0.10520492379272695 1000 9 110.11111111111111
0 0.11046101595541845 1000 10 99.0
0 0.12479055604795981 1000 11 89.9090909090909
0 0.13641409066666377 1000 12 82.33333333333333
0 0.16597902557157865 1000 13 75.92307692307692
0 0.1603930312139094 1000 14 70.42857142857143
0 0.18519530407211318 1000 15 65.66666666666667
0 0.19365201195346263 1000 16 61.5
0 0.23241874330160806 1000 17 57.8235294117647
0 0.23024766906165892 1000 18 54.55555555555556
0 0.2961820739110923 1000 19 51.63157894736842
0 0.2877457513541644 1000 20 49.0
0 0.2791568207231402 1000 21 46.61904761904762
0 0.2733734261467636 1000 22 44.45454545454545
0 0.30909458002777784 1000 23 42.47826086956522
0 0.325

7 0.0 1000 5 199.0
7 0.0 1000 6 165.66666666666666
7 0.0 1000 7 141.85714285714286
7 0.0 1000 8 124.0
7 0.0 1000 9 110.11111111111111
7 0.0 1000 10 99.0
7 0.0 1000 11 89.9090909090909
7 0.0 1000 12 82.33333333333333
7 0.0 1000 13 75.92307692307692
7 0.0 1000 14 70.42857142857143
7 0.0 1000 15 65.66666666666667
7 0.0 1000 16 61.5
7 0.0 1000 17 57.8235294117647
7 0.0 1000 18 54.55555555555556
7 0.0 1000 19 51.63157894736842
7 0.0 1000 20 49.0
7 0.0 1000 21 46.61904761904762
7 0.0 1000 22 44.45454545454545
7 0.0 1000 23 42.47826086956522
7 0.0 1000 24 40.666666666666664
7 0.0 1000 25 39.0
7 0.0 1000 26 37.46153846153846
7 0.0 1000 27 36.03703703703704
7 0.0 1000 28 34.714285714285715
7 0.0 1000 29 33.48275862068966
7 0.0 1000 30 32.333333333333336
7 0.0 1000 31 31.258064516129032
7 0.0 1000 32 30.25
7 0.0 1000 33 29.303030303030305
7 0.0 1000 34 28.41176470588235
7 0.0 1000 35 27.571428571428573
7 0.0 1000 36 26.77777777777778
7 0.0 1000 37 26.027027027027028
7 0.0 1000 38 25.315789473684

13 0.0 1000 2 499.0
13 0.0 1000 3 332.3333333333333
13 0.0 1000 4 249.0
13 0.0 1000 5 199.0
13 0.0 1000 6 165.66666666666666
13 0.0 1000 7 141.85714285714286
13 0.0 1000 8 124.0
13 0.0 1000 9 110.11111111111111
13 0.0 1000 10 99.0
13 0.0 1000 11 89.9090909090909
13 0.0 1000 12 82.33333333333333
13 0.0 1000 13 75.92307692307692
13 0.0 1000 14 70.42857142857143
13 0.0 1000 15 65.66666666666667
13 0.0 1000 16 61.5
13 0.0 1000 17 57.8235294117647
13 0.0 1000 18 54.55555555555556
13 0.0 1000 19 51.63157894736842
13 0.0 1000 20 49.0
13 0.0 1000 21 46.61904761904762
13 0.0 1000 22 44.45454545454545
13 0.0 1000 23 42.47826086956522
13 0.0 1000 24 40.666666666666664
13 0.0 1000 25 39.0
13 0.0 1000 26 37.46153846153846
13 0.0 1000 27 36.03703703703704
13 0.0 1000 28 34.714285714285715
13 0.0 1000 29 33.48275862068966
13 0.0 1000 30 32.333333333333336
13 0.0 1000 31 31.258064516129032
13 0.0 1000 32 30.25
13 0.0 1000 33 29.303030303030305
13 0.0 1000 34 28.41176470588235
13 0.0 1000 35 27.5714285

22 0.0 1000 8 124.0
22 0.0 1000 9 110.11111111111111
22 0.0 1000 10 99.0
22 0.0 1000 11 89.9090909090909
22 0.0 1000 12 82.33333333333333
22 0.0 1000 13 75.92307692307692
22 0.0 1000 14 70.42857142857143
22 0.0 1000 15 65.66666666666667
22 0.0 1000 16 61.5
22 0.0 1000 17 57.8235294117647
22 0.0 1000 18 54.55555555555556
22 0.0 1000 19 51.63157894736842
22 0.0 1000 20 49.0
22 0.0 1000 21 46.61904761904762
22 0.0 1000 22 44.45454545454545
22 0.0 1000 23 42.47826086956522
22 0.0 1000 24 40.666666666666664
22 0.0 1000 25 39.0
22 0.0 1000 26 37.46153846153846
22 0.0 1000 27 36.03703703703704
22 0.0 1000 28 34.714285714285715
22 0.0 1000 29 33.48275862068966
22 0.0 1000 30 32.333333333333336
22 0.0 1000 31 31.258064516129032
22 0.0 1000 32 30.25
22 0.0 1000 33 29.303030303030305
22 0.0 1000 34 28.41176470588235
22 0.0 1000 35 27.571428571428573
22 0.0 1000 36 26.77777777777778
22 0.0 1000 37 26.027027027027028
22 0.0 1000 38 25.31578947368421
22 0.0 1000 39 24.641025641025642
22 0.0 1000 40 

In [9]:
print('Dxx  =','{0: >#014.10f}'.format(np.mean(Dfxx)),'{0: >#014.10f}'.format(np.std(Dfxx)/np.sqrt(nrep)))
print('Dyy  =','{0: >#014.10f}'.format(np.mean(Dfyy)),'{0: >#014.10f}'.format(np.std(Dfyy)/np.sqrt(nrep)))
print('Dxy  =','{0: >#014.10f}'.format(np.mean(Dfxy)),'{0: >#014.10f}'.format(np.std(Dfxy)/np.sqrt(nrep)))
print('-----------------------')
print('Dx2  =','{0: >#014.10f}'.format(np.mean(Dfx2d)),'{0: >#014.10f}'.format(np.std(Dfx2d)/np.sqrt(nrep)))
print('Dy2  =','{0: >#014.10f}'.format(np.mean(Dfy2d)),'{0: >#014.10f}'.format(np.std(Dfy2d)/np.sqrt(nrep)))
print('D2d  =','{0: >#014.10f}'.format(np.mean(Df2d)),'{0: >#014.10f}'.format(np.std(Df2d)/np.sqrt(nrep)))

Dxx  =   0.0797886159   0.0449833903
Dyy  =   0.0704507122   0.0395965699
Dxy  =  -0.0079876208   0.0109665543
-----------------------
Dx2  =   0.0715786554   0.0412176921
Dy2  =   0.0786606726   0.0462505651
D2d  =   0.0751196640   0.0415081024


In [10]:
for item in Dfxx:
    print(item)

0.8363642936702692
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.40294577498347534
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.6756167118306525
0.0
0.0
0.0
